In [1]:
!pip3 install gensim
!pip3 install nltk

  Using cached https://files.pythonhosted.org/packages/86/f3/37504f07651330ddfdefa631ca5246974a60d0908216539efda842fd080f/gensim-3.5.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a8/0b/f163da98d3a01b3e0ef1cab8dd2123c34aee2bafbb1c5bffa354cc8a1730/scipy-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/88/29/f4c845648ed23264e986cdc5fbab5f8eace1be5e62144ef69ccc7189461d/numpy-1.15.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/23/10/c0b78c27298029e4454a472a1919bde20cb182dab1662cec7f2ca1dcc523/boto-2.49.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/ad/b7775a16329acfbf6eecf0198c05caa8a067c518fd84728b3aaad60446ea/boto3-1.7.65-py2.py3-none-any.whl
  Using cached https://files.pyt

In [2]:
# coding: utf-8

import time
import warnings
from collections import Counter
from gensim.models import Word2Vec
import pandas as pd
import scipy
import time
from nltk.corpus import stopwords
import numpy as np
from KaggleWord2VecUtility import KaggleWord2VecUtility
from numpy import float32
import math
from sklearn.ensemble import RandomForestClassifier
import sys
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
from sklearn.cluster import KMeans
from sklearn.svm import SVC, LinearSVC
import pickle
from math import *
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize

warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)


/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def clustering(word_vectors,test_word_vectors,num_clusters):
    clf=KMeans(n_clusters=num_clusters, random_state=0)
    clf.fit(word_vectors)
    idx=clf.predict(word_vectors)
    idx_test=clf.predict(test_word_vectors)
    #idx_proba=clf.predict_proba(word_vectors)
    centroids=clf.cluster_centers_
    return (idx,idx_test,centroids)

In [3]:
def getdocvec(review,model,features):
    wordlist=review.split()
    #print (len(wordlist))
    docvec=np.zeros([len(wordlist),features], dtype="float32")
    i=0
    for word in wordlist:
        if(word in model.wv.vocab):
            #print (word)
            docvec[i]=model[word]
            i+=1
    #print (pca)
    #docvec=(pca.fit_transform(docvec.transpose())).transpose()
    docvec=np.asmatrix(docvec)
    #print (docvec.shape)
    return docvec

In [4]:
num_features = 200 #int(sys.argv[1])     # Word vector dimensionality
min_word_count = 20   # Minimum word count
num_workers = 40       # Number of threads to run in parallel
context = 10          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words
sub_dim=3   #dimension for subspaces
model_name = str(num_features) + "features_" + str(min_word_count) + "minwords_" + str(context) + "context_len2alldata"
# Load the trained Word2Vec model. 
model = Word2Vec.load(model_name)
# Get wordvectors for all words in vocabulary.
word_vectors = model.wv.vectors
# Load train data.
train = pd.read_csv( 'data/train_v2.tsv', header=0, delimiter="\t")
# Load test data.
test = pd.read_csv( 'data/test_v2.tsv', header=0, delimiter="\t")
all = pd.read_csv( 'data/all_v2.tsv', header=0, delimiter="\t")
# Set number of clusters.
num_clusters = 40 #int(sys.argv[2])

In [5]:
traindata = []
for i in range( 0, len(all["news"])):
	traindata.append(" ".join(KaggleWord2VecUtility.review_to_wordlist(all["news"][i], True)))
print (traindata[0])
print (all.shape)

lerxst wam umd edu thing subject car nntp posting host rac wam umd edu organization university maryland college park lines wondering anyone could enlighten car saw day door sports car looked late early called bricklin doors really small addition front bumper separate rest body know anyone tellme model name engine specs years production car made history whatever info funky looking car please mail thanks il brought neighborhood lerxst
(18846, 2)


In [6]:
tfv = TfidfVectorizer(strip_accents='unicode',dtype=np.float32)
tfidfmatrix_traindata = tfv.fit_transform(traindata)
featurenames = tfv.get_feature_names()
idf = tfv._tfidf.idf_
# Creating a dictionary with word mapped to its idf value 
print ("Creating word-idf dictionary for Training set...")

word_idf_dict = {}
for pair in zip(featurenames, idf):
	word_idf_dict[pair[0]] = pair[1]

Creating word-idf dictionary for Training set...


In [7]:
print (all.shape,train.shape,test.shape)

(18846, 2) (11314, 3) (7532, 3)


In [8]:
#converting training data into 4x200 subspaces
doc4vecs=np.zeros((len(train["news"]),sub_dim,num_features))
doccovrep=np.zeros((len(train["news"]),num_features,num_features))
pca=PCA(sub_dim) #pca object for dimensionality reduction
docvec=[]
for i in range(0,len(train["news"])):
    docvec=getdocvec(train["news"][i],model,num_features)
    #converting to 4x200 subspace
    doc4vecs[i]=(pca.fit_transform(docvec.transpose())).transpose()
    #doccovrep[i]=np.cov(doc4vecs[i].transpose())
    #doc4vecs[i]=docvec
    if(i%1000==0):
        print ("Train reviews covered:",i)
        print (doc4vecs[i].shape)
        #print (doccovrep[i].shape)
print ("done")

Train reviews covered: 0
(3, 200)
Train reviews covered: 1000
(3, 200)
Train reviews covered: 2000
(3, 200)
Train reviews covered: 3000
(3, 200)
Train reviews covered: 4000
(3, 200)
Train reviews covered: 5000
(3, 200)
Train reviews covered: 6000
(3, 200)
Train reviews covered: 7000
(3, 200)
Train reviews covered: 8000
(3, 200)
Train reviews covered: 9000
(3, 200)
Train reviews covered: 10000
(3, 200)
Train reviews covered: 11000
(3, 200)
done


In [9]:
#converting test data into 4x200 subspaces,idx_proba.shape
doc4vecs_test=np.zeros((len(test["news"]),sub_dim,num_features))
doccovrep_test=np.zeros((len(test["news"]),num_features,num_features))

docvec=[]
for i in range(0,len(test["news"])):
    docvec=getdocvec(test["news"][i],model,num_features)
    #converting to 4x200 subspace
    doc4vecs_test[i]=(pca.fit_transform(docvec.transpose())).transpose()
    #doccovrep_test[i]=np.cov(doc4vecs_test[i].transpose())

    #doc4vecs_test[i]=docvec
    if(i%1000==0):
        print ("Test reviews covered:",i)
        print (doc4vecs_test[i].shape)
        #print (doccovrep[i].shape)
print ("done")

Test reviews covered: 0
(3, 200)
Test reviews covered: 1000
(3, 200)
Test reviews covered: 2000
(3, 200)
Test reviews covered: 3000
(3, 200)
Test reviews covered: 4000
(3, 200)
Test reviews covered: 5000
(3, 200)
Test reviews covered: 6000
(3, 200)
Test reviews covered: 7000
(3, 200)
done


IMPLEMENTING COVARIANCE METHOD

getting 4x4 or 3x3 cov matrix for each subspace, therefore for each document.
Then we flatten it to dim 10 or 6

In [11]:
#getting mean vector for training subspaces
mean_doc_vecs=np.zeros((len(doc4vecs),num_features))
for i in range(0,len(doc4vecs)):
    for j in range(0,sub_dim):
        mean_doc_vecs[i]+=doc4vecs[i][j] #add 200d of each dimension of subspace
        j+=1
    mean_doc_vecs/=sub_dim #averaging
    if(i%1000==0):
        print("Training subspaces made mean: ",i)
        print(mean_doc_vecs[i].shape)
    i+=1


Training subspaces made mean:  0
(200,)
Training subspaces made mean:  1000
(200,)
Training subspaces made mean:  2000
(200,)
Training subspaces made mean:  3000
(200,)
Training subspaces made mean:  4000
(200,)
Training subspaces made mean:  5000
(200,)
Training subspaces made mean:  6000
(200,)
Training subspaces made mean:  7000
(200,)
Training subspaces made mean:  8000
(200,)
Training subspaces made mean:  9000
(200,)
Training subspaces made mean:  10000
(200,)
Training subspaces made mean:  11000
(200,)


In [10]:
#covariance matrix + flattening for training subspaces
cov_dim=int((num_features*(num_features+1))/2)
print (cov_dim)
covrep=np.zeros((len(doc4vecs),cov_dim))
for i in range(0,len(doc4vecs)):
    #finding covariance matrix
    cov=np.cov(doc4vecs[i].transpose())
    #flattening
    a=0
    for p in range(0,num_features):
        for q in range(p,num_features):
            if(p==q):
                covrep[i][a]=cov[p][q]
            else:
                covrep[i][a]=cov[p][q]*np.sqrt(2)
            a+=1
            q+=1
        p+=1
    if(i%1000==0):
        print (i)
        print (covrep[i].shape)
    i+=1
    
print (covrep.shape)

20100
0
(20100,)
1000
(20100,)
2000
(20100,)
3000
(20100,)
4000
(20100,)
5000
(20100,)
6000
(20100,)
7000
(20100,)
8000
(20100,)
9000
(20100,)
10000
(20100,)
11000
(20100,)
(11314, 20100)


In [11]:
#getting mean vector for test subspaces

In [12]:
#covariance matrix + flattening for test subspaces
print (cov_dim)
covrep_test=np.zeros((len(doc4vecs_test),cov_dim))
for i in range(0,len(doc4vecs_test)):
    #finding covariance matrix
    cov=np.cov(doc4vecs_test[i].transpose())
    #flattening
    a=0
    for p in range(0,num_features):
        for q in range(p,num_features):
            if(p==q):
                covrep_test[i][a]=cov[p][q]
            else:
                covrep_test[i][a]=cov[p][q]*np.sqrt(2)
            a+=1
            q+=1
        p+=1
    if(i%1000==0):
        print (i)
        print (covrep_test[i].shape)
    i+=1
    
print (covrep_test.shape)

20100
0
(20100,)
1000
(20100,)
2000
(20100,)
3000
(20100,)
4000
(20100,)
5000
(20100,)
6000
(20100,)
7000
(20100,)
(7532, 20100)


In [13]:
print (scipy.sparse.issparse(covrep_test))

False


In [ ]:
#clustering word vectors
#trainingvecs=np.zeros((sub_dim*len(doc4vecs),num_features))
trainingvecs=np.zeros((num_features*len(doc4vecs),num_features)) #for cov rep
j=0
k=0
for i in range(0,num_features*len(doc4vecs)):
    trainingvecs[i]=doccovrep[j][k]
    k+=1 #move on to next set of 200 features
    k=k%num_features #%sub_dim
    i+=1 #next 1x200 vector sample
    if(k==0):
        j+=1 #move on to next doc
    if(i%30000==0):
        print ("Training vectors gathered: ",i)
print (trainingvecs.shape)
#print (doc4vecs[1][2][0:3])
#print (trainingvecs[6][0:3]) #confirming that vectors were stored correctly

In [ ]:
#clustering word vectors
#testvecs=np.zeros((sub_dim*len(doc4vecs_test),num_features))
testvecs=np.zeros((num_features*len(doc4vecs_test),num_features))
j=0
k=0
for i in range(0,num_features*len(doc4vecs_test)):
    testvecs[i]=doccovrep_test[j][k]
    k+=1 #move on to next set of 200 features
    k=k%num_features #%sub_dim
    i+=1 #next 1x200 vector sample
    if(k==0):
        j+=1 #move on to next set of doc
    if(i%3000==0):
        print ("Test vectors gathered: ",i)
print (testvecs.shape)
#print (doc4vecs_test[1][2][0:3])
#print (testvecs[6][0:3]) #confirming that vectors were stored correctly

In [ ]:
#clustering these vectors
idx,idx_test,centroids=clustering(trainingvecs,testvecs,num_clusters)
print (idx.shape, centroids.shape)
print (Counter(idx))

In [ ]:
#DIRECTLY CONCATENATING 4x200 VECTORS
train_con4features=np.zeros((len(train),sub_dim*num_features))
for i in range(0,len(doc4vecs)):
    for j in range(0,4):
        for k in range(0,num_features):
            train_con4features[i][(j*num_features)+k]=doc4vecs[i][j][k]
            k+=1
        j+=1
    i+=1
print (train_con4features.shape)
#for test dataset
test_con4features=np.zeros((len(test),sub_dim*num_features))
for i in range(0,len(doc4vecs_test)):
    for j in range(0,4):
        for k in range(0,num_features):
            test_con4features[i][(j*num_features)+k]=doc4vecs_test[i][j][k]
            k+=1
        j+=1
    i+=1
print (test_con4features.shape)

In [ ]:
#NOT DIRECTLY CONCATENATING, BUT AVERAGING VECTORS [not averaging, just adding]
#SO EACH DOCUMENT HAS 1X200 REP, AVERAGE OF ITS 4
train_avg4features=np.zeros((len(train),num_features))
for i in range(0,len(doc4vecs)):
    for j in range(0,4):
        train_avg4features[i]+=doc4vecs[i][j] #adding all 4 vectors of doc number i
        j+=1
    #train_avg4features/=4 #averaging
    if (i%3000==0):
        print (train_avg4features[i][0:2])  
    i+=1
print (train_avg4features.shape)

test_avg4features=np.zeros((len(test),num_features))
for i in range(0,len(doc4vecs_test)):
    for j in range(0,4):
        if (i%3000==0):
            print (test_avg4features[i][0:2],doc4vecs_test[i][j][0:2])
            print (test_avg4features[i][0:2]+doc4vecs_test[i][j][0:2])
        test_avg4features[i]+=doc4vecs_test[i][j] #adding all 4 vectors of doc number i
        j+=1
    #test_avg4features/=4 #averaging
    if (i%3000==0):
        print (test_avg4features[i][0:2])  
    i+=1
print (test_avg4features.shape)
print (test_avg4features[0:5])

In [ ]:
#creating training input vector
train_mat=np.zeros((len(doc4vecs),num_clusters*num_features))
print (train_mat.shape)
j=0
k=0
#trainingvecs[0][0]
#print (train_mat[0])
for i in range(0,len(trainingvecs)):
    #print (i,j,idx[i])
    vec_clus=int(idx[i])
    for k in range(0,num_features):
    #print (idx[i])
        train_mat[j][(num_features*vec_clus)+k] += trainingvecs[i][k]
        #print (train_mat[j][(200*vec_clus)+k],trainingvecs[i][k])
    i+=1
    j=int(i/sub_dim)
    
print (idx[0:4])
#print (trainingvecs[0])
#print (train_mat[0][8000:8200]==trainingvecs[0])

In [ ]:
#creating test input vector
j=0
k=0
test_mat=np.zeros((len(doc4vecs_test),num_clusters*num_features))
print (test_mat.shape)    
for i in range(0,len(testvecs)):
    #print (i,j,idx[i])
    vec_clus=int(idx_test[i])
    for k in range(0,num_features):
    #print (idx[i])
        test_mat[j][(num_features*vec_clus)+k] += testvecs[i][k]
        #print (train_mat[j][(200*vec_clus)+k],trainingvecs[i][k])
    i+=1
    j=int(i/sub_dim)
print (idx_test[0:4])
#print (test_mat[0][200:400]==testvecs[0])
#print (train["class"])

In [14]:
#saving feature matrices
test_name = "TEST_SUB_" + str(num_clusters) + "cluster_" + str(num_features) + "feature_matrix_kmeans_sparse.npy"
train_name = "SUB_" + str(num_clusters) + "cluster_" + str(num_features) + "feature_matrix_kmeans_sparse.npy"
np.save(train_name, train_mat)
np.save(test_name, test_mat)

NameError: name 'train_mat' is not defined

In [ ]:
clf = LinearSVC()
clf.fit(covrep, train["class"])
Y_true, Y_pred  = test["class"], clf.predict(covrep_test)
print (classification_report(Y_true, Y_pred, digits=6))

In [ ]:
#applying classifier
start=time.time()
param_grid = [{'C': np.arange(0.1, 4.0, 0.6)}]
scores = ['f1_weighted']#,'accuracy', 'recall_micro', 'f1_micro' , 'precision_micro', 'recall_macro', 'f1_macro' , 'precision_macro', 'recall_weighted', 'precision_weighted','f1_weighted'] #, 'accuracy', 'recall', 'f1']
for score in scores:
	strt = time.time()
	print ("# Tuning hyper-parameters for", score, "\n", "Running with ",num_clusters," clusters")
	clf = GridSearchCV(LinearSVC(C=1), param_grid, cv=3, scoring= '%s' % score)
	#clf = LinearSVC()
	clf.fit(covrep, train["class"])
	endtime = time.time()
	print ("Best parameters set found on development set:\n")
	print (clf.best_params_)
	print ("Best value for ", score, ":\n")
	print (clf.best_score_)
	Y_true, Y_pred  = test["class"], clf.predict(covrep_test)
	print ("Report")
	print (classification_report(Y_true, Y_pred, digits=6))
	print ("Accuracy: ",clf.score(covrep_test,test["class"]))
	print ("Time taken:", endtime - start, "\n")

# Tuning hyper-parameters for f1_weighted 
 Running with  40  clusters


In [ ]:
print (doc4vecs.shape)
print (trainingvecs.shape)
print (train_avg4features.shape)
print (train_con4features.shape)
print (train_mat.shape)